In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertModel
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset, DataLoader, Subset
import numpy as np

# Load the Excel file and check for NaN values
file_path = 'train_data.xlsx'
df = pd.read_excel(file_path)
df = df.dropna(subset=['Label_Sentiment'])  # Remove rows with NaN in 'Label_Sentiment'
df['Label_Sentiment'] = df['Label_Sentiment'].astype(int)  # Ensure Label_Sentiment is integer type

class TextDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = self.dataframe.iloc[idx, 1]
        tokens = self.tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors="pt")
        label = torch.tensor(self.dataframe.iloc[idx, 2], dtype=torch.long)
        
        sample = {
            'input_ids': tokens['input_ids'].squeeze(),
            'attention_mask': tokens['attention_mask'].squeeze(),
            'label': label
        }
        return sample

# Define the dataset
dataset = TextDataset(dataframe=df)

# Define the Text-Only Model with BERT
class TextOnlyModel(nn.Module):
    def __init__(self):
        super(TextOnlyModel, self).__init__()
        self.text_model = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(p=0.3)
        self.classifier = nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask):
        text_outputs = self.text_model(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state[:, 0, :]
        text_outputs = self.dropout(text_outputs)
        logits = self.classifier(text_outputs)
        return logits

# Initialize model, loss, and optimizer
model = TextOnlyModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# K-Fold Cross-Validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
fold_results = []

for fold, (train_idx, test_idx) in enumerate(skf.split(dataset, df['Label_Sentiment'])):
    print(f'Fold {fold + 1}')
    
    train_subsampler = Subset(dataset, train_idx)
    test_subsampler = Subset(dataset, test_idx)
    
    train_dataloader = DataLoader(train_subsampler, batch_size=16, shuffle=True)
    test_dataloader = DataLoader(test_subsampler, batch_size=16, shuffle=False)
    
    # Training loop with early stopping
    model.train()
    for epoch in range(10):  
        epoch_loss = 0.0
        for batch in train_dataloader:
            optimizer.zero_grad()
            outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
            loss = criterion(outputs, batch['label'])
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
        # Validation step
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in test_dataloader:
                outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
                loss = criterion(outputs, batch['label'])
                val_loss += loss.item()
        
        print(f'Epoch {epoch + 1} - Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}')
    
    # Evaluation
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for batch in test_dataloader:
            outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
            _, preds = torch.max(outputs, 1)
            all_labels.extend(batch['label'].numpy())
            all_preds.extend(preds.numpy())

    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, zero_division=1)
    precision = precision_score(all_labels, all_preds, zero_division=1)
    recall = recall_score(all_labels, all_preds, zero_division=1)
    conf_matrix = confusion_matrix(all_labels, all_preds, labels=[0, 1])

    fold_results.append({
        'fold': fold + 1,
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'confusion_matrix': conf_matrix
    })
    print(f'Fold {fold + 1} - Accuracy: {accuracy}, F1: {f1}, Precision: {precision}, Recall: {recall}')
    print(f'Confusion Matrix:\n{conf_matrix}')

# Average results across folds
avg_accuracy = np.mean([result['accuracy'] for result in fold_results])
avg_f1 = np.mean([result['f1'] for result in fold_results])
avg_precision = np.mean([result['precision'] for result in fold_results])
avg_recall = np.mean([result['recall'] for result in fold_results])

print(f'Average Accuracy: {avg_accuracy}')
print(f'Average F1 Score: {avg_f1}')
print(f'Average Precision: {avg_precision}')
print(f'Average Recall: {avg_recall}')

/home/tipu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Fold 1
Epoch 1 - Train Loss: 239.7554, Val Loss: 59.0254
Epoch 2 - Train Loss: 117.0425, Val Loss: 23.8161
Epoch 3 - Train Loss: 38.2449, Val Loss: 17.4169
Epoch 4 - Train Loss: 18.2665, Val Loss: 19.8774
Epoch 5 - Train Loss: 14.0185, Val Loss: 18.5096
Epoch 6 - Train Loss: 16.4520, Val Loss: 25.6166
Epoch 7 - Train Loss: 19.2055, Val Loss: 61.3749
Epoch 8 - Train Loss: 23.3493, Val Loss: 37.9496
Epoch 9 - Train Loss: 230.8779, Val Loss: 61.6892
Epoch 10 - Train Loss: 245.9910, Val Loss: 61.7501
Fold 1 - Accuracy: 0.4971711456859972, F1: 0.6641473783656117, Precision: 0.4971711456859972, Recall: 1.0
Confusion Matrix:
[[  0 711]
 [  0 703]]
Fold 2
Epoch 1 - Train Loss: 246.7692, Val Loss: 61.6849
Epoch 2 - Train Loss: 245.8023, Val Loss: 61.6857
Epoch 3 - Train Loss: 245.8849, Val Loss: 61.7122
Epoch 4 - Train Loss: 245.6570, Val Loss: 61.6848
Epoch 5 - Train Loss: 245.7600, Val Loss: 61.7459
Epoch 6 - Train Loss: 245.6190, Val Loss: 61.6885
Epoch 7 - Train Loss: 245.6987, Val Loss: 61

In [3]:
# Save the trained model
torch.save(model.state_dict(), 'text_only_model.pth')  # Save only the weights


In [4]:
# Load the test data
test_file_path = 'test_data.xlsx'
test_df = pd.read_excel(test_file_path)
test_df = test_df.dropna(subset=['Label_Sentiment'])  # Ensure no NaN values
test_df['Label_Sentiment'] = test_df['Label_Sentiment'].astype(int)  # Ensure integer labels

# Update Dataset class for testing
class TestTextDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = self.dataframe.iloc[idx, 1]  # Update column index if necessary
        tokens = self.tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors="pt")
        label = torch.tensor(self.dataframe.iloc[idx, 2], dtype=torch.long)
        
        sample = {
            'input_ids': tokens['input_ids'].squeeze(),
            'attention_mask': tokens['attention_mask'].squeeze(),
            'label': label
        }
        return sample

# Define the test dataset
test_dataset = TestTextDataset(dataframe=test_df)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Load the trained model
trained_model = TextOnlyModel()
trained_model.load_state_dict(torch.load('text_only_model.pth'))  # Update path if necessary
trained_model.eval()

# Evaluate on test data
all_labels = []
all_preds = []

with torch.no_grad():
    for batch in test_dataloader:
        outputs = trained_model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
        _, preds = torch.max(outputs, 1)
        all_labels.extend(batch['label'].numpy())
        all_preds.extend(preds.numpy())

# Calculate metrics
accuracy = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds, zero_division=1)
precision = precision_score(all_labels, all_preds, zero_division=1)
recall = recall_score(all_labels, all_preds, zero_division=1)
conf_matrix = confusion_matrix(all_labels, all_preds, labels=[0, 1])

print(f'Test Accuracy: {accuracy}')
print(f'Test F1 Score: {f1}')
print(f'Test Precision: {precision}')
print(f'Test Recall: {recall}')
print(f'Confusion Matrix:\n{conf_matrix}')


/tmp/ipykernel_865185/114247217.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  trained_model.load_state_dict(torch.load('text_only_model.pth'))  # Update path if neces

Test Accuracy: 0.4855769230769231
Test F1 Score: 0.0
Test Precision: 1.0
Test Recall: 0.0
Confusion Matrix:
[[606   0]
 [642   0]]
